# Lab 4 - Model operationalization

In the previous lab, we used `Automated ML` to select the optimal model and registered the best performing version of the model in Model Registry.

Now, we are ready to deploy the model.

The model can be deployed to a variety of target runtimes, including:
- Azure Container Instance
- Azure Kubernetes Service
- IoT Edge
- FPGA


In this lab, we will deploy the model as a web service in Azure Container Instance.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/amlarch.png)

In [16]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.74


## Connect to the workspace

In [17]:
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: /data/home/demouser/notebooks/MTC_AzureAILabs/DataScienceTrack/01-aml-walkthrough/aml_config/config.json


## Deploy as web service

To build the correct environment for ACI, provide the following:
* A scoring script that invokes the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before




###  Retrieve the model
The model is registered in the workspace's Model Registry. As you recall we added a couple of properties to the model during registration. We will use *FriendlyName* property to retrieve the model.


In [3]:
from azureml.core.model import Model

# Retrieve the model based on "FriendlyName" tag
models = Model.list(ws, properties=[["FriendlyName", "Lab3BestModel"]])

for model in models:
    print("Model name:", model.name)
    
model = models[0]

Model name: AutoMLac5fd8f37best
Model name: AutoML448799fabbest


### Create scoring script

Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:
* The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.

In [4]:
%%writefile score.py
import json
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.applications import resnet50

from azureml.core.model import Model
import azureml.train.automl

def init():
    # Instantiate ResNet50 featurizer
    global featurizer
    featurizer = resnet50.ResNet50(
            weights = 'imagenet', 
            input_shape=(224,224,3), 
            include_top = False,
            pooling = 'avg')

    # Load the model
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path(model_name = '<<modelid>>')
    model = joblib.load(model_path)
  

def run(raw_data):
    try:
        # convert json to numpy array
        images = np.array(json.loads(raw_data)['data'])
        # normalize as required by ResNet50
        images = resnet50.preprocess_input(images.astype(float))
        # extract bottleneck features
        features = featurizer.predict(images)
        # make prediction
        predictions = model.predict(features)
        # Add string labels
        labels = ["Barren",
                  "Cultivated",
                  "Developed",
                  "Forest",
                  "Herbaceous",
                  "Shrub"]
        
        # Get string labels for predictions
        string_predictions = [labels[pred] for pred in predictions]
        
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    
    # Return both numeric and string predictions
    return json.dumps({"predictions": predictions.tolist(), "labels": string_predictions})

Writing score.py


Substitute the actual *model ID* in the script file.

In [5]:
script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()
    
with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelid>>', model.name))

### Create environment file
Next, create an environment file that specifies the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `sklearn`, `numpy`,  `tensorflow`, `h5py`  and Azure ML SDK dependencies.

To ensure the consistency of the prediction results with the training results, the AML SDK dependency versions used by the scoring environment needs to be the same as in the environment that was used to train the model.

The SDK dependency versions used to train the model can be retrieved from the run history.

You need to replace the values in `experiment_name` and `run_id` variables with the name of your experiment and the run ID number saved in the previous lab.


In [6]:
from azureml.core import Experiment
from azureml.train.automl.run import AutoMLRun

# Retrieve training run record
experiment_name = 'aerial-automatedML'
run_id = model.properties['RunID'] 

experiment = Experiment(ws, experiment_name)
automl_run = AutoMLRun(experiment = experiment, run_id = run_id)

# Retrieve SDK dependencies. 
dependencies = automl_run.get_run_sdk_dependencies(iteration = 0, check_versions=False)
print(dependencies)

{'azureml-train-automl': '0.1.74', 'azureml-telemetry': '0.1.74', 'azureml-defaults': '0.1.74', 'azureml-dataprep': '0.3.0', 'azureml-dataprep-native': '10.4.0', 'azureml-core': '0.1.74'}


We can now create an environment file.

In [7]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
#myenv.add_channel('defaults')
#myenv.add_pip_package("azureml-sdk[notebooks,automl]==" + sdk_version)
#myenv.remove_pip_package('azureml-defaults')
for package, version in dependencies.items():
    myenv.add_pip_package('{}=={}'.format(package, version))
    
myenv.add_pip_package("scikit-learn==0.19.1")
myenv.add_pip_package("numpy==1.14.5")
myenv.add_pip_package("h5py==2.8.0")
myenv.add_pip_package("pynacl==1.2.1")
myenv.add_tensorflow_pip_package(core_type='cpu')

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of `myenv.yml` file

In [8]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl==0.1.74
  - azureml-telemetry==0.1.74
  - azureml-defaults==0.1.74
  - azureml-dataprep==0.3.0
  - azureml-dataprep-native==10.4.0
  - azureml-core==0.1.74
  - scikit-learn==0.19.1
  - numpy==1.14.5
  - h5py==2.8.0
  - pynacl==1.2.1
  - tensorflow==1.10.0



### Create docker image for deployment

In [9]:
from azureml.core.image import ContainerImage, Image

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml",
                                                  description="Image for aerial classifier",
                                                  tags={"Classifier": "AutomatedML"})

image = Image.create(name = "aerial-best-automl-classifier",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running................................................
SucceededImage creation operation finished for image aerial-best-automl-classifier:1, operation "Succeeded"


### Define ACI configuration

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. The default is 1 core and 1 gigabyte of RAM. Since we are using ResNet50 featurizer we are CPU bound.  In this lab we will use the defaults but you should always go through the proper performance plannig exercise to find the right configuration.

In [10]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "aerial",  "method" : "classifier"}, 
                                               description='Predict aerial images')

### Deploy in ACI

Deploy the image as a web service in Azure Container Instance.

In [11]:
from azureml.core.webservice import Webservice

aci_service_name = 'aerial-automl-svc'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

aerial-automl-svc
Creating service
Running........................................
SucceededACI service creation operation finished, operation "Succeeded"


Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [12]:
print(aci_service.scoring_uri)

http://13.68.179.234:80/score


## Test deployed service

You will now test the deployed model with 3 test images.  
 
First, download sample images.

In [13]:
%%sh
mkdir samples
cd samples
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/barren-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/cultivated-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/developed-1.png
ls -l


total 248
-rw-rw-r-- 1 demouser demouser 71190 Nov  6 01:19 barren-1.png
-rw-rw-r-- 1 demouser demouser 79841 Nov  6 01:19 cultivated-1.png
-rw-rw-r-- 1 demouser demouser 96252 Nov  6 01:19 developed-1.png


2018-11-20 16:58:06 URL:https://azureailabs.blob.core.windows.net/aerialsamples/barren-1.png [71190/71190] -> "barren-1.png" [1]
2018-11-20 16:58:07 URL:https://azureailabs.blob.core.windows.net/aerialsamples/cultivated-1.png [79841/79841] -> "cultivated-1.png" [1]
2018-11-20 16:58:07 URL:https://azureailabs.blob.core.windows.net/aerialsamples/developed-1.png [96252/96252] -> "developed-1.png" [1]


Define utility function that wraps loading images and invoking the service. The function takes as an input a list of pathnames to images. It loads and converts the binary images into a JSON array and invokes the service passing the JSON payload.

In [18]:
from PIL import Image
import numpy as np
import json

def score(pathnames, service):
    images = []
    for pathname in pathnames:
        img = Image.open(pathname)
        img = np.asarray(img).tolist()
        images.append(img)
    images = json.dumps({"data": images})
    images = bytes(images, encoding='utf8')
    results = json.loads(service.run(input_data=images))
    return results

Call the service.

In [19]:
aci_service = Webservice(workspace=ws, name='aerial-automl-svc')

pathnames = ['samples/barren-1.png', 'samples/developed-1.png', 'samples/cultivated-1.png']

results = score(pathnames, aci_service)

print(results)

{'labels': ['Barren', 'Developed', 'Cultivated'], 'predictions': [0, 2, 1]}


## Clean up resources

Delete the ACI service.

In [ ]:
aci_service.delete()